In [1]:
import pandas as pd
import json
from datasets import Dataset, concatenate_datasets
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
from unsloth.chat_templates import get_chat_template, train_on_responses_only

# ✅ 1. 读取 Excel 数据集
xlsx_path = "/root/autodl-tmp/result_with_accuracy.xlsx"
df_xlsx = pd.read_excel(xlsx_path, usecols=["Disease", "Symptoms", "Treatments"])
df_xlsx.fillna("", inplace=True)  # ✅ 处理缺失值

dataset_xlsx = Dataset.from_pandas(df_xlsx)

# ✅ 2. 读取 JSON 数据集
json_path = "/root/autodl-tmp/updated_data.json"
with open(json_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

formatted_qa = []
formatted_textbook = []

for item in json_data:
    if item["type"] == "qa":
        user_input = f"<|start_header_id|>user<|end_header_id|>\n\n{item['question']}\n\n"
        assistant_response = f"<|start_header_id|>assistant<|end_header_id|>\n\n{item['answer']}\n\n"
        formatted_qa.append({"text": user_input + assistant_response})
    
    elif item["type"] == "textbook":
        assistant_response = f"<|start_header_id|>assistant<|end_header_id|>\n\n{item['text']}\n\n"
        formatted_textbook.append({"text": assistant_response})

dataset_qa = Dataset.from_list(formatted_qa)
dataset_textbook = Dataset.from_list(formatted_textbook)

# ✅ 3. 处理 Excel 数据
def formatting_prompts_func(examples):
    formatted_texts = []
    for disease, symptoms, treatments in zip(examples["Disease"], examples["Symptoms"], examples["Treatments"]):
        user_input = f"<|start_header_id|>user<|end_header_id|>\n\n{disease}\n\n"
        assistant_response = f"<|start_header_id|>assistant<|end_header_id|>\n\n"
        assistant_response += f"Symptoms:\n{symptoms}\n\n" if symptoms else ""
        assistant_response += f"Treatments:\n{treatments}\n\n" if treatments else ""
        formatted_texts.append(user_input + assistant_response)
    return {"text": formatted_texts}

dataset_xlsx = dataset_xlsx.map(formatting_prompts_func, batched=True)

# ✅ 4. 合并数据集
dataset_combined = concatenate_datasets([dataset_xlsx, dataset_qa, dataset_textbook])

# ✅ 5. 加载 `unsloth/Llama-3.1-8B-Instruct-bnb-4bit`
model_name = "/root/autodl-tmp/Llama-3.1-8B-Instruct-bnb-4bit"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

# ✅ 6. 应用 `LoRA` 训练参数（减少影响）
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # ⬇️ 降低 LoRA 适配影响（从 16 降到 8）
    target_modules=["q_proj", "v_proj", "o_proj"],  # ⬇️ 只适配这三个关键层
    lora_alpha=8,  # ⬇️ 降低 LoRA 权重（从 16 降到 8）
    lora_dropout=0.05,  # 适当加一点 dropout 防止过拟合
    bias="none",
    use_gradient_checkpointing="unsloth",
)

# 设定对话模板
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# ✅ 7. 训练器（减少学习率影响）
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_combined,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,  # ⬆️ 增大 batch size，减少梯度波动
        gradient_accumulation_steps=1,  # ⬇️ 降低累计梯度，减少 LoRA 影响
        warmup_steps=10,  # 适当增加 warmup
        max_steps=500,  # ⬇️ 减少训练步数，避免过拟合
        learning_rate=5e-5,  # ⬇️ 降低学习率，减少对原始模型的影响
        fp16=False,  
        bf16=True,  
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # ✅ 改为 `cosine` 让学习率平滑下降
        seed=3407,
        output_dir="trained_model",
        report_to="none",
    ),
)

# ✅ 8. 训练仅关注 `assistant` 回答部分
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

# ✅ 9. 开始训练
trainer.train()

# ✅ 10. 保存模型
trainer.model.save_pretrained("/root/autodl-tmp/trained_model")
trainer.tokenizer.save_pretrained("/root/autodl-tmp/trained_model")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Map:   0%|          | 0/410 [00:00<?, ? examples/s]

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.643 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Map (num_proc=2):   0%|          | 0/276485 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/276485 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 276,485 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 500
 "-____-"     Number of trainable parameters = 5,505,024


Step,Training Loss
1,2.737000
2,2.505000
3,2.622500
4,2.336900
5,2.353800
6,2.263400
7,2.246400
8,2.110000
9,2.617400
10,2.196000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/root/autodl-tmp/trained_model/tokenizer_config.json',
 '/root/autodl-tmp/trained_model/special_tokens_map.json',
 '/root/autodl-tmp/trained_model/tokenizer.json')